In [1]:
import emoji

In [2]:
emoji.EMOJI_ALIAS_UNICODE

{':1st_place_medal:': '🥇',
 ':2nd_place_medal:': '🥈',
 ':3rd_place_medal:': '🥉',
 ':AB_button_(blood_type):': '🆎',
 ':ATM_sign:': '🏧',
 ':A_button_(blood_type):': '🅰',
 ':Afghanistan:': '🇦🇫',
 ':Albania:': '🇦🇱',
 ':Algeria:': '🇩🇿',
 ':American_Samoa:': '🇦🇸',
 ':Andorra:': '🇦🇩',
 ':Angola:': '🇦🇴',
 ':Anguilla:': '🇦🇮',
 ':Antarctica:': '🇦🇶',
 ':Antigua_&_Barbuda:': '🇦🇬',
 ':Aquarius:': '♒',
 ':Argentina:': '🇦🇷',
 ':Aries:': '♈',
 ':Armenia:': '🇦🇲',
 ':Aruba:': '🇦🇼',
 ':Ascension_Island:': '🇦🇨',
 ':Australia:': '🇦🇺',
 ':Austria:': '🇦🇹',
 ':Azerbaijan:': '🇦🇿',
 ':BACK_arrow:': '🔙',
 ':B_button_(blood_type):': '🅱',
 ':Bahamas:': '🇧🇸',
 ':Bahrain:': '🇧🇭',
 ':Bangladesh:': '🇧🇩',
 ':Barbados:': '🇧🇧',
 ':Belarus:': '🇧🇾',
 ':Belgium:': '🇧🇪',
 ':Belize:': '🇧🇿',
 ':Benin:': '🇧🇯',
 ':Bermuda:': '🇧🇲',
 ':Bhutan:': '🇧🇹',
 ':Bolivia:': '🇧🇴',
 ':Bosnia_&_Herzegovina:': '🇧🇦',
 ':Botswana:': '🇧🇼',
 ':Bouvet_Island:': '🇧🇻',
 ':Brazil:': '🇧🇷',
 ':British_Indian_Ocean_Territory:': '🇮🇴',
 ':British_Virgin_Is

In [3]:
emoji_dict = {
    '0':'\u2764\uFE0F',
    '1':':basketball:',
    '2':':grinning_face_with_big_eyes:',
    '3':':disappointed_face:',
    '4':':fork_and_knife:'
}

In [4]:
print(emoji.emojize(':fire:'), end= '\n\n')
for e in emoji_dict.values():
    print(emoji.emojize(e))

🔥

❤️
🏀
😃
😞
🍴


In [5]:
import pandas as pd
import numpy as np

In [6]:
train = pd.read_csv('train_emoji.csv', header=None)
test = pd.read_csv('test_emoji.csv', header=None)

In [7]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [8]:
test.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a raise\t,2
3,she got me a present\t,0
4,ha ha ha it was so funny\t,2


In [9]:
Xtrain = train[0].values
Ytrain = train[1].values

In [10]:

for i in range(5):
    print(Xtrain[i],emoji.emojize(emoji_dict[str(Ytrain[i])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


In [11]:
Xtest = test[0]
Ytest = test[1]

In [12]:
! ls

Untitled.ipynb
best_model.h5
glove.6B.50d.txt
test_emoji.csv
train_emoji.csv


In [13]:
f = open('glove.6B.50d.txt', encoding='utf8')

In [14]:
embedding_index = {}
for line in f :
    values = line.split()
    word = values[0]
    coef = np.asarray(values[1:], dtype = 'float')
    embedding_index[word] = coef

In [15]:
f.close()

In [16]:
emb_dim = embedding_index['eat'].shape[0]

# convert sentence into vectors (embedding layer output)

In [17]:
def embedding_output(X):
    maxLen = 10
    embedding_out = np.zeros((X.shape[0],maxLen, emb_dim))
    
    for ix in range(X.shape[0]):
        
        X[ix] = X[ix].split()
        
        for ij in range(len(X[ix])):
            try:
                embedding_out[ix][ij] = embedding_index[X[ix][ij].lower()] 
            except:
                embedding_out[ix][ij] = np.zeros((50,))
                
    return embedding_out

In [18]:
embedding_out_train = embedding_output(Xtrain)
embedding_out_test = embedding_output(Xtest)

c:\users\manan mongia\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [19]:
print(Xtrain[0])
print((len(Xtrain[0])))

['never', 'talk', 'to', 'me', 'again']
5


In [20]:
print(embedding_out_train.shape)
print(embedding_out_test.shape)

(132, 10, 50)
(56, 10, 50)


In [21]:
from keras.utils import to_categorical
Y_train = to_categorical(Ytrain,num_classes=5)
Y_test = to_categorical(Ytest,num_classes=5)

Using TensorFlow backend.


# Create RNN/LSTM Model

In [22]:
from keras.models import Sequential
from keras.layers import *

In [39]:
model = Sequential()
model.add(LSTM(64,input_shape = (10,50), return_sequences = True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences = False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer= 'adam',metrics = ['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_9 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_6 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________________________


In [40]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
earlystop = EarlyStopping(patience=20, monitor='val_acc')
checkpoint = ModelCheckpoint('best_model.h5',monitor='val_acc', verbose=True, save_best_only=True)
# del earlystop
# del checkpoint

model.fit(embedding_out_train,Y_train,
          epochs=100,
          batch_size=64,
          shuffle=True,
          validation_split=0.15
          ,callbacks=[earlystop, checkpoint]
         )

Train on 112 samples, validate on 20 samples
Epoch 1/100
112/112 [==============================] - 3s 31ms/step - loss: 1.6024 - acc: 0.1964 - val_loss: 1.6057 - val_acc: 0.2500

Epoch 00001: val_acc improved from -inf to 0.25000, saving model to best_model.h5
Epoch 2/100
112/112 [==============================] - 0s 678us/step - loss: 1.5837 - acc: 0.2054 - val_loss: 1.6072 - val_acc: 0.3000

Epoch 00002: val_acc improved from 0.25000 to 0.30000, saving model to best_model.h5
Epoch 3/100
112/112 [==============================] - 0s 576us/step - loss: 1.5447 - acc: 0.3036 - val_loss: 1.6190 - val_acc: 0.2000

Epoch 00003: val_acc did not improve from 0.30000
Epoch 4/100
112/112 [==============================] - 0s 750us/step - loss: 1.5206 - acc: 0.3750 - val_loss: 1.6375 - val_acc: 0.2000

Epoch 00004: val_acc did not improve from 0.30000
Epoch 5/100
112/112 [==============================] - 0s 725us/step - loss: 1.5210 - acc: 0.3304 - val_loss: 1.6535 - val_acc: 0.2000

Epoch 000

112/112 [==============================] - 0s 716us/step - loss: 0.2355 - acc: 0.9286 - val_loss: 0.8469 - val_acc: 0.6500

Epoch 00043: val_acc did not improve from 0.70000
Epoch 44/100
112/112 [==============================] - 0s 748us/step - loss: 0.2474 - acc: 0.9196 - val_loss: 0.7753 - val_acc: 0.7000

Epoch 00044: val_acc did not improve from 0.70000
Epoch 45/100
112/112 [==============================] - 0s 748us/step - loss: 0.1837 - acc: 0.9732 - val_loss: 0.9341 - val_acc: 0.6500

Epoch 00045: val_acc did not improve from 0.70000
Epoch 46/100
112/112 [==============================] - 0s 712us/step - loss: 0.2277 - acc: 0.9643 - val_loss: 1.2061 - val_acc: 0.6000

Epoch 00046: val_acc did not improve from 0.70000
Epoch 47/100
112/112 [==============================] - 0s 734us/step - loss: 0.2492 - acc: 0.9107 - val_loss: 1.0312 - val_acc: 0.6500

Epoch 00047: val_acc did not improve from 0.70000
Epoch 48/100
112/112 [==============================] - 0s 823us/step - loss: 

In [38]:
#before EarlyStopping
model.evaluate(embedding_out_test,Y_test)

56/56 [==============================] - 0s 286us/step


[1.0995064888681685, 0.5535714285714286]

In [41]:
# After EarlyStopping
model.load_weights('best_model.h5')
model.evaluate(embedding_out_test,Y_test)

56/56 [==============================] - 0s 468us/step


[1.6206331082752772, 0.6250000085149493]

In [42]:
model.evaluate(embedding_out_train,Y_train)

132/132 [==============================] - 0s 288us/step


[0.13972393643449654, 0.9696969696969697]

In [43]:
pred = model.predict_classes(embedding_out_test)
pred

array([4, 3, 3, 3, 2, 2, 1, 2, 4, 2, 1, 2, 3, 0, 1, 3, 3, 2, 3, 4, 3, 0,
       4, 3, 3, 1, 2, 0, 1, 2, 0, 1, 3, 2, 0, 1, 2, 3, 4, 2, 1, 0, 0, 1,
       2, 2, 3, 2, 3, 3, 1, 0, 3, 2, 3, 3], dtype=int64)

In [50]:
for i in range(30):
    print(' '.join(Xtest[i]))
    print('Actual: ',emoji.emojize(emoji_dict[str(Ytest[i])]))
    print('prediction: ',emoji.emojize(emoji_dict[str(Ytest[i])]),end ='\n\n')
    

I want to eat
Actual:  🍴
prediction:  🍴

he did not answer
Actual:  😞
prediction:  😞

he got a raise
Actual:  😃
prediction:  😞

she got me a present
Actual:  ❤️
prediction:  😞

ha ha ha it was so funny
Actual:  😃
prediction:  😃

he is a good friend
Actual:  ❤️
prediction:  😃

I am upset
Actual:  ❤️
prediction:  🏀

We had such a lovely dinner tonight
Actual:  ❤️
prediction:  😃

where is the food
Actual:  🍴
prediction:  🍴

Stop making this joke ha ha ha
Actual:  😃
prediction:  😃

where is the ball
Actual:  🏀
prediction:  🏀

work is hard
Actual:  😞
prediction:  😃

This girl is messing with me
Actual:  😞
prediction:  😞

are you serious ha ha
Actual:  😃
prediction:  ❤️

Let us go play baseball
Actual:  🏀
prediction:  🏀

This stupid grader is not working
Actual:  😞
prediction:  😞

work is horrible
Actual:  😞
prediction:  😞

Congratulation for having a baby
Actual:  😃
prediction:  😃

stop messing around
Actual:  😞
prediction:  😞

any suggestions for dinner
Actual:  🍴
prediction:  🍴

I love ta

In [52]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Ytest, pred)

array([[ 5,  1,  4,  2,  0],
       [ 0,  5,  0,  0,  0],
       [ 3,  2,  9,  4,  0],
       [ 0,  2,  2, 11,  0],
       [ 0,  0,  0,  1,  5]], dtype=int64)

In [53]:
emoji_dict

{'0': '❤️',
 '1': ':basketball:',
 '2': ':grinning_face_with_big_eyes:',
 '3': ':disappointed_face:',
 '4': ':fork_and_knife:'}